In [ ]:
import numpy as np
import pandas as pd

In [ ]:
import nltk
nltk.download()

In [ ]:
from nltk.corpus import  names
from nltk.stem import WordNetLemmatizer

In [ ]:
# train, test set 불러오기

df = pd.read_csv("TRAIN.csv")
df.test = pd.read_csv("TEST.csv")

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  after removing the cwd from sys.path.


In [ ]:
# train set 확인
df.head()

,Unnamed: 0,User,Post,Label,LabelNumber
0,5,user-5,"['Since I dont know what DBT is, would you min...",Supportive,4.0
1,116,user-116,['I didnt... I always feel as though Im a fail...,Ideation,2.0
2,45,user-45,['Thanks for this. I was working with my dad t...,Ideation,2.0
3,16,user-16,"['oh, doh.I had someone tell me similar recent...",Ideation,2.0
4,462,user-462,['I know the feeling of not wanting to be a bu...,Ideation,2.0


In [ ]:
# test set 확인
df.test.head()

,Post
0,['Your dad loves you. Call him and tell him yo...
1,['Please believe what this person said. I can...
2,['I feel youre Ache pal. My world feels Hypera...
3,['I dont do much currently. I lifeguard for a ...
4,"['you always can kill yourself later:)', 'no n..."


In [ ]:
# remove number and punctuation 

def letters_only(word):
  return word.isalpha()

In [ ]:
# remove name entity

from nltk.corpus import names
all_names = set(names.words())

In [ ]:
# lemmaization

from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [ ]:
# 대문자를 소문자로 변환
# 숫자, 기호, 이름 제거

def clean_text(doc):
  cleaned_doc = []
  for word in doc.split(' '): 
    word = word.lower() # ABD -> abd
  
    if letters_only(word) and word not in all_names and len(word) > 2: # remove number and punc. and name entity
      cleaned_doc.append(lemmatizer.lemmatize(word))
  return ' '.join(cleaned_doc) 

cleaned_post1 = [clean_text(post) for post in df.Post]

In [ ]:
# 변환 잘 되었는지 확인
cleaned_post1

In [ ]:
# stopword 제거

from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 

stop_words = set(stopwords.words('english'))
stop_words.update(['arent', 'didnt', 'couldnt', 'cant', 'doesnt', 'dont', 'hes', 'hadnt', 'hasnt', 'havent', 'isnt', 'mightnt',
                   'mustnt', 'neednt', 'shant', 'shes', 'shouldve', 'shouldnt', 'thatll', 'wasnt', 'werent', 'wont', 'wouldnt', 
                   'youd', 'youll', 'youre', 'youve', 'would', 'maybe', 'might'])

def rm_stopwords(doc):
  result = []
  doc_tokens = word_tokenize(doc)
  for w in doc_tokens:
    if w not in stop_words:
      result.append(w)
  return ' '.join(result)

cleaned_post2 = [rm_stopwords(post) for post in cleaned_post1]

In [ ]:
# 제거 잘 되었는지 확인
cleaned_post2

In [ ]:
# train set에 변환한 post 입력
# 잘 바뀌었는지 확인

df.Post = cleaned_post2
df.head()

,Unnamed: 0,User,Post,Label,LabelNumber
0,5,user-5,know dbt mind explaining little bit really sor...,Supportive,4.0
1,116,user-116,always feel though failure say going scare kno...,Ideation,2.0
2,45,user-45,wa working dad today grandma house took sat th...,Ideation,2.0
3,16,user-16,someone tell similar took calling someone prom...,Ideation,2.0
4,462,user-462,know feeling wanting burden rough night last n...,Ideation,2.0


In [ ]:
# subset 구성 후 tuple로 변환
# tuple로 잘 변환되었는지 확인

subset = df[['Post', 'LabelNumber']]
tuples = [tuple(x) for x in subset.values]
data = tuples
data

In [ ]:
# count_words 함수 정의
# tuple 데이터를 변수로 받음
# 각 word가 5 가지 classes(attempt, behavior, ideation, indicator, supportive)에서 몇 번 나오는지 세는 함수
# 출력은 set
def count_words(training_set):
  counts_dict = {}
  for post, labelnumber in training_set:
    for word in word_tokenize(post):
      if word not in counts_dict:
        counts_dict[word] = [0, 0, 0, 0, 0]
      if labelnumber == 0:
        counts_dict[word][0] += 1
      elif labelnumber == 1:
        counts_dict[word][1] += 1
      elif labelnumber == 2:
        counts_dict[word][2] += 1
      elif labelnumber == 3:
        counts_dict[word][3] += 1
      else:
        counts_dict[word][4] += 1
  return counts_dict

In [ ]:
# 각 단어 별로 몇 번 나왔는지 확인

word_counting = count_words(data)
word_counting

In [ ]:
# word_counting 접근 방법
# 'survive' 단어가 (attempt, behavior, ideation, indicator, supportive) 순서로 몇 번 나왔는지
word_counting['survive']

[1, 5, 7, 8, 10]

In [ ]:
# piror 확률을 계산하기 위해 각 class 개수 세기

counts_attempt = 0
counts_behavior = 0
counts_ideation = 0
counts_indicator = 0
counts_supportive = 0

for i in range(0, len(df)):
  if df.Label[i] == 'Attempt':
    counts_attempt += 1
  elif df.Label[i] == 'Behavior':
    counts_behavior += 1
  elif df.Label[i] == 'Ideation':
    counts_ideation += 1
  elif df.Label[i] == 'Indicator':
    counts_indicator += 1
  else:
    counts_supportive += 1

In [ ]:
# 각 class 별 개수를 전체 개수로 나누어 prior 확률 계산

prob_attempt = counts_attempt / len(df)
prob_behavior = counts_behavior / len(df)
prob_ideation = counts_ideation / len(df)
prob_indicator = counts_indicator / len(df)
prob_supportive = counts_supportive / len(df)

In [ ]:
# prior 확률을 list로 만들어줌
# 그 후 확인
prob_prior = [prob_attempt, prob_behavior, prob_ideation, prob_indicator, prob_supportive]
prob_prior

[0.07428571428571429,
 0.15428571428571428,
 0.3657142857142857,
 0.20285714285714285,
 0.20285714285714285]

In [ ]:
# likelihood 계산하기 위해 train set의 각 class 별 단어 개수 세기

total_counts_byclass = [0, 0, 0, 0, 0]
for word in word_counting:
  for i in range(0,5):
   total_counts_byclass[i] += word_counting[word][i]

In [ ]:
# train set의 각 class 별 단어 개수 확인
# 하지만, 이것만으로는 부족
# 그 이유는 test set에만 있고, train set에는 없는 단어가 있을 수 있기 때문
# 따라서, test set에만 있는 단어들을 역시 계산해서 포함시켜 줘야함
# 그러한 단어들은 아직 class가 없기 때문에 기존에 있던 class들의 가중치(비율)에 따라 더해줘야 함

total_counts_byclass

[16459, 26083, 47915, 36201, 38032]

In [ ]:
# 기존 class들의 가중치

counts_ratio = [0, 0, 0, 0, 0]
for i in range(0,5):
  counts_ratio[i] = round((total_counts_byclass[i] / sum(total_counts_byclass)),2)

counts_ratio

[0.1, 0.16, 0.29, 0.22, 0.23]

In [ ]:
# 현재 train set의 중복되지 않는 단어 개수
len(word_counting)

10436

In [ ]:
# test set에만 있는 단어들을 찾기 위해 test set 전처리

cleaned_test1 = [clean_text(post) for post in df.test.Post]
cleaned_test2 = [rm_stopwords(post) for post in cleaned_test1]
df.test.Post = cleaned_test2
df.test

In [ ]:
# set을 만들어 중복을 없앰
# train set의 단어 모음

b = set()
for post in df.Post:
  a = word_tokenize(post)
  for w in a:
    b.update([w])

In [ ]:
# len(word_counting)의 결과값과 같은 것 확인

len(b)

10436

In [ ]:
# test set의 단어들을 추가시킴
# 똑같은 방법으로 set을 사용하여 중복 없앰

for post in df.test.Post:
  c = word_tokenize(post)
  for w in c:
    b.update([w])

In [ ]:
# 단어의 개수가 늘어난 것 확인
len(b)

12308

In [ ]:
# test set에만 있는 단어의 개수

12306-10436

1870

In [ ]:
# 각 class 별 가중치로 1870개를 배분함

w_counts_byclass = [0, 0, 0, 0, 0]
for i in range(0, 5):
  w_counts_byclass[i] = total_counts_byclass[i] + round(1870 * counts_ratio[i])

w_counts_byclass

[16646, 26382, 48457, 36612, 38462]

In [ ]:
# likelihood 계산 (이대로 파라미터 조정)
# 각 단어의 likelihood를 각 class 별로 계산 후 set으로 묶음
# Laplace smoothing 적용 : 모든 단어가 한 번씩은 출현했다고 보고, 분모에는 모든 단어 개수(train + test)를 더함

def get_likelihood(count_dict, smoothing = 1):
  likelihood={}
  for word in count_dict:
    likelihood[word] = [0, 0, 0, 0, 0]
    for i in range(0,5):
      likelihood[word][i] = (count_dict[word][i] + smoothing) / (w_counts_byclass[i] + len(b))
  return likelihood

In [ ]:
# likelihood 계산 후 확인

prob_likelihood = get_likelihood(word_counting, smoothing = 1)
prob_likelihood

In [ ]:
# classifier 개발

def classifier(test_set, likelihood, prior):
  # test set 전 처리
  cleaned_test1 = [clean_text(post) for post in test_set.Post]
  cleaned_test2 = [rm_stopwords(post) for post in cleaned_test1]
  test_set.Post = cleaned_test2
 
  result = []
  na_count = 0
  for post in test_set.Post:
    log_prior = np.log(prior)
    tokenword = word_tokenize(post)
    for i in range(0,5):
      for w in tokenword:
        if w in likelihood:
          log_prior[i] = log_prior[i] + np.log(likelihood[w][i]) # log 취해서 더 해줌. 왜냐하면 그대로 두면 값이 너무 작아짐
    result.append(np.where(log_prior == max(log_prior))[0][0])  # 각 class 별로 계산된 확률 중 높은 값의 인덱스를 확인
  result2 = np.array(result)
  return result2

In [ ]:
final = classifier(df.test, prob_likelihood, prob_prior)
final

array([4, 4, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 4, 2, 2, 2, 2, 2,
       2, 2, 2, 4, 2, 2, 2, 2, 4, 2, 2, 2, 2, 2, 2, 2, 2, 4, 2, 2, 4, 2,
       2, 2, 4, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 4, 4, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 4, 2, 2, 2, 4, 4, 2, 2, 4, 2, 4, 4, 4, 2, 2, 2, 2,
       2, 2, 4, 2, 3, 4, 2, 4, 4, 4, 2, 4, 2, 2, 2, 2, 2, 2, 4, 2, 2, 4,
       2, 2, 4, 2, 2, 2, 2, 2, 4, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 4, 2,
       4, 2, 2, 4, 2, 2, 2, 2, 1, 4, 2, 2, 2, 2, 4, 2, 2, 2])